In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# keras imports for the dataset and building our neural network
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
import pandas as pd
# to calculate accuracy
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
train = pd.read_csv('/content/drive/My Drive/mnist-idx/mnist_train-1.csv')
test = pd.read_csv('/content/drive/My Drive/mnist-idx/mnist_test-1.csv')
#print(train.head())
x_train, x_validation, y_train, y_validation = train_test_split(train.iloc[:,1:],
    train.iloc[:,0],
    test_size=0.2,
    random_state=42,
    stratify=train.iloc[:,0])

# We'll create a "split" of training, for cross-validation
train_images_split = []
train_labels_split = []
validation_images = []
validation_labels = []

# We'll create a full training set
train_images_full = []
train_labels_full = []

# And the test set
test_images = []

for index, row in x_train.iterrows():
    train_images_split.append(row.values[0:].reshape((28, 28)))
    train_labels_split.append(row.iloc[0])
    train_images_full.append(row.values[0:].reshape((28, 28)))
    train_labels_full.append(row.iloc[0])

for index, row in x_validation.iterrows():
    validation_images.append(row.values[0:].reshape((28, 28)))
    validation_labels.append(row.iloc[0])
    train_images_full.append(row.values[0:].reshape((28, 28)))
    train_labels_full.append(row.iloc[0])

for index, row in test.iterrows():
    test_images.append(row.iloc[1:].values.reshape((28, 28)))

# Convert numpy array, while normalizing the image data
train_labels_split = np.array(train_labels_split)
train_images_split = np.array(train_images_split) / 255.
validation_labels = np.array(validation_labels)
validation_images = np.array(validation_images) / 255.
train_labels_full = np.array(train_labels_full)
train_images_full = np.array(train_images_full) / 255.
test_images = np.array(test_images) / 255.



In [8]:
train_images = np.expand_dims(train_images_split, axis=3)
validation_images = np.expand_dims(validation_images, axis=3)

In [9]:
# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(1,1)))
# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(100, activation='relu'))
# output layer
model.add(Dense(10, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')



In [10]:
train_gen = ImageDataGenerator().flow(train_images, train_labels_split, batch_size=32)
validation_gen = ImageDataGenerator().flow(validation_images, validation_labels, batch_size=32)
checkpointer = keras.callbacks.ModelCheckpoint(filepath = r'/content/drive/My Drive/small_train/checkpoint-mnist-random-intialisation.hdf5',
                                               monitor='accuracy',
                                               mode='min',
                                               save_best_only=False,
                                               verbose=1)

In [12]:
train_images.shape

(47999, 28, 28, 1)

In [13]:
# Model fit
history = model.fit(train_gen,
                    validation_data = validation_gen,
                    steps_per_epoch = len(train_images) / 32,
                    validation_steps = len(validation_images) / 32,
                    epochs=5
                    )

Epoch 1/5


ValueError: ignored

In [15]:
# keras imports for the dataset and building our neural network
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

# to calculate accuracy
from sklearn.metrics import accuracy_score

# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# building the input vector from the 28x28 pixels
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(1,1)))
# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(100, activation='relu'))
# output layer
model.add(Dense(10, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
checkpointer = keras.callbacks.ModelCheckpoint(filepath = r'/content/drive/My Drive/small_train/checkpoint-mnist-random-intialisation.hdf5',
                                               monitor='accuracy',
                                               mode='min',
                                               save_best_only=False,
                                               verbose=1)
# training the model for 10 epochs
model.fit(X_train, Y_train, batch_size=128, epochs=5, validation_data=(X_test, Y_test),callbacks=[checkpointer])

Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)
Epoch 1/5
469/469 [==============================] - 4s 9ms/step - loss: 0.3652 - accuracy: 0.8915 - val_loss: 0.0769 - val_accuracy: 0.9761

Epoch 00001: saving model to /content/drive/My Drive/small_train/checkpoint-mnist-random-intialisation.hdf5
Epoch 2/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0597 - accuracy: 0.9831 - val_loss: 0.0600 - val_accuracy: 0.9807

Epoch 00002: saving model to /content/drive/My Drive/small_train/checkpoint-mnist-random-intialisation.hdf5
Epoch 3/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0318 - accuracy: 0.9905 - val_loss: 0.0564 - val_accuracy: 0.9819

Epoch 00003: saving model to /content/drive/My Drive/small_train/checkpoint-mnist-random-intialisation.hdf5
Epoch 4/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0198 - accuracy: 0.9940 - val_loss: 0.0603 - val_accuracy: 0.9815

Epoch 00004: saving mode